In [18]:
import torch 
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
) 

We pass the Dataset as an argument to `DataLoader`. This wraps an iterable over our dataset, and supports automatic `batching`, `sampling`, `shuffling` and `multiprocess data loading`.

In [10]:
batch_size = 64
# create the dataloaders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
for x, y in test_dataloader:
    print("The shape of the x [N,C,H,W] is : {}". format(x.shape))
    print("The shape of Y : {}". format(y.shape))
    break

The shape of the x [N,H,W,C] is : torch.Size([64, 1, 28, 28])
The shape of Y : torch.Size([64])


In [15]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
device

device(type='mps')

In [20]:
#defining the model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # initialises the nn.Module
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In a single training loop, the model should make predictions on the training data (fed to it in batches), and backpropagate the prediction error to adjust the model’s parameters

In [25]:
# defining the training loop

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x,y) in enumerate(dataloader): 
        x,y = x.to(device), y.to(device)
        
        pred = model(x)
        
        loss = loss_fn(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()
        
        if batch % 100 == 0 :
            print(f" The loss is {loss.item()} and number of training samples processsed : {(batch + 1)* len(x)}")


In [ ]:
def test(dataloader, model, loss_fn):
    
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0 
    
    with torch.no_grad(): # Turns off gradient computation (for faster inference and less memory usage)
        
        for batch, (x,y) in enumerate(dataloader):
        
            x, y = x.to(device), y.to(device)
            
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  #.item() is called on a single valued tensor
            
    test_loss /= num_batches 
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1} \n")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Complete")

Epoch 1 

 The loss is 2.300150156021118 and number of training samples processsed : 64
 The loss is 0.5691792368888855 and number of training samples processsed : 6464
 The loss is 0.3898773789405823 and number of training samples processsed : 12864
 The loss is 0.49439969658851624 and number of training samples processsed : 19264
 The loss is 0.4421764314174652 and number of training samples processsed : 25664
 The loss is 0.4317706823348999 and number of training samples processsed : 32064
 The loss is 0.37308767437934875 and number of training samples processsed : 38464
 The loss is 0.522210419178009 and number of training samples processsed : 44864
 The loss is 0.4747474491596222 and number of training samples processsed : 51264
 The loss is 0.5466564893722534 and number of training samples processsed : 57664
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.425684 

Epoch 2 

 The loss is 0.2752690613269806 and number of training samples processsed : 64
 The loss is 0.37370142340660095 

# Saving the Model
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [28]:
torch.save(model.state_dict(),"model")

## Loading the Model
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [32]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model",weights_only= True))

<All keys matched successfully>

This model can now be used to make predictions.

In [33]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [38]:
# testing on an example

model.eval()
x, y = test_data[0][0],test_data[0][1]
with torch.no_grad():
    x=x.to(device)
    pred = model(x)
    predicted, actual = classes[pred.argmax()], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
